## Simulating Data

In [1]:
#######################################################################
#    Helper Functions
#######################################################################

def generate_cement_data(start_year=1980, end_year=1998):
    """
    Generate synthetic cement industry data matching Ryan (2012) Table 2 including population data
    """
    import numpy as np
    from scipy import stats
    import pandas as pd
    
    years = list(range(start_year, end_year + 1))
    n_years = len(years)
    n_markets = 27
    
    # Initialize empty lists
    data = []
    
    # Generate base market populations (in millions)
    # Mean around 10M with reasonable variation across markets
    market_base_pop = stats.truncnorm.rvs(
        (0.7 - 10.0) / 7.0,  # min around 0.7M
        (33.0 - 10.0) / 7.0,  # max around 33M
        loc=10.0,  # mean 10M
        scale=7.0,  # SD 7M
        size=n_markets
    )
    
    # Annual population growth rates (mean 1.5% with variation)
    market_growth_rates = stats.truncnorm.rvs(
        (0.005 - 0.015) / 0.005,
        (0.025 - 0.015) / 0.005,
        loc=0.015,  # mean 1.5%
        scale=0.005,  # SD 0.5%
        size=n_markets
    )
    
    # Target average of 4.75 plants per market (from Table 2)
    plants_per_market = {m: np.random.choice([3,4,5,6], p=[0.15, 0.35, 0.35, 0.15]) 
                        for m in range(n_markets)}
    
    plant_id = 0
    
    for market in range(n_markets):
        n_plants = plants_per_market[market]
        base_pop = market_base_pop[market]
        growth_rate = market_growth_rates[market]
        
        for plant in range(n_plants):
            # Generate base capacity matching Table 2 statistics
            base_capacity = stats.truncnorm.rvs(
                (196 - 797) / 386,  # min
                (2678 - 797) / 386,  # max
                loc=797,  # mean
                scale=386,  # size
                size=1
            )[0]
            
            for year in years:
                if year == start_year:
                    capacity = base_capacity
                else:
                    # 10% chance of significant investment/divestment
                    if np.random.random() < 0.1:
                        investment = stats.norm.rvs(2.19, 77.60)
                        capacity = prev_capacity + investment
                    else:
                        # Small random changes
                        capacity = prev_capacity * (1 + stats.norm.rvs(0, 0.01))
                
                # Calculate population for this market-year
                years_from_start = year - start_year
                population = base_pop * (1 + growth_rate)**years_from_start * 1_000_000  # Convert to actual population
                
                # Ensure capacity stays within bounds
                capacity = np.clip(capacity, 196, 2678)
                
                # Generate quantity as 85-90% of capacity with some noise
                utilization_rate = stats.truncnorm.rvs(
                    (0.85 - 0.875) / 0.025,
                    (0.90 - 0.875) / 0.025,
                    loc=0.875,
                    scale=0.025
                )
                quantity = capacity * utilization_rate
                
                # Calculate investment
                investment = capacity - prev_capacity if year > start_year else 0
                
                # Add random large investments with low probability
                if np.random.random() < 0.01:  # 1% chance
                    investment = stats.norm.rvs(loc=0, scale=300)
                
                data.append({
                    'year': year,
                    'market': market,
                    'plant_id': plant_id,
                    'capacity': round(capacity),
                    'quantity': round(quantity),
                    'investment': round(investment),
                    'post_1990': 1 if year >= 1990 else 0,
                    'population': round(population)
                })
                
                prev_capacity = capacity
            
            plant_id += 1
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    
    # Add market-level variables
    market_year = df.groupby(['market', 'year']).agg({
        'capacity': 'sum',
        'quantity': 'sum',
        'plant_id': 'count',
        'population': 'first'  # Population is same for all plants in market-year
    }).reset_index()
    
    market_year.columns = ['market', 'year', 'market_capacity', 'market_quantity', 'n_plants', 'market_population']
    
    # Merge market data back
    df = df.merge(market_year, on=['market', 'year'])
    
    # Calculate competitors' capacity
    df['competitors_capacity'] = df['market_capacity'] - df['capacity']
    
    return df


######################################
# Data Generation
######################################

df = generate_cement_data()

# Print summary statistics
print("\nPlant-level Summary Statistics:")
print(df[['capacity', 'quantity', 'investment']].describe())

print("\nMarket-level Population Summary Statistics (in millions):")
print(df.groupby(['market', 'year'])['population'].first().div(1_000_000).describe())

print("\nAverage plants per market:", 
      df.groupby(['market', 'year'])['plant_id'].count().mean())

# Save to CSV
df.to_csv('cement_data.csv', index=False)


Plant-level Summary Statistics:
          capacity     quantity   investment
count  2299.000000  2299.000000  2299.000000
mean    790.775120   692.140496     0.173554
std     312.707274   273.555985    42.551866
min     196.000000   167.000000  -702.000000
25%     525.500000   460.000000    -5.000000
50%     771.000000   677.000000     0.000000
75%    1014.000000   888.500000     5.000000
max    1783.000000  1585.000000   770.000000

Market-level Population Summary Statistics (in millions):
count    513.000000
mean      13.597147
std        6.522427
min        1.514546
25%       10.186491
50%       13.535461
75%       18.089433
max       29.322071
Name: population, dtype: float64

Average plants per market: 4.481481481481482


## Tables 6 & 7: Investment Policy Functions (Adjustment Band Size and Target Level)

In [5]:
#######################################################################
# Package Imports and Helper Functions
#######################################################################
import numpy as np
import pandas as pd 
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import SplineTransformer, StandardScaler
import warnings
warnings.filterwarnings('ignore')



def estimate_investment_policy(df_input, spec="I", period="Pre-1990"):
    """
    Estimate investment policy function following Ryan (2012) Tables 6-7
    Returns both band and target equation results
    """
    df = df_input.copy()
    
    # Split by period
    if period == "Pre-1990":
        df = df[df['year'] < 1990]
    else:
        df = df[df['year'] >= 1990]
        
    # Create capacity change variables 
    df['capacity_change'] = df.groupby('plant_id')['capacity'].diff()
    df['log_capacity'] = np.log(df['capacity'])
    
    # Filter to substantial changes (>5% change in capacity)
    changes = df[np.abs(df['capacity_change']/df['capacity']) > 0.05].copy()
    changes['log_change'] = np.log(np.abs(changes['capacity_change']))
    
    # Normalize capacities based on specification
    per_capita = spec in ["III", "IV"]
    if per_capita:
        changes['capacity_norm'] = changes['capacity'] / (changes['population'] / 10_000_000)
        changes['competitors_capacity_norm'] = changes['competitors_capacity'] / (changes['population'] / 10_000_000)
    else:
        changes['capacity_norm'] = changes['capacity'] / 1_000_000  # Convert to millions of tons
        changes['competitors_capacity_norm'] = changes['competitors_capacity'] / 1_000_000
        
    # Create spline transformer
    spline_transform = SplineTransformer(
        n_knots=8,  # Fewer knots
        degree=3,
        knots='quantile'  # Use quantile knots
    )
    
    # Scale variables before creating splines
    scaler = StandardScaler()
    own_scaled = scaler.fit_transform(changes['capacity_norm'].values.reshape(-1, 1))
    comp_scaled = scaler.fit_transform(changes['competitors_capacity_norm'].values.reshape(-1, 1))
    
    # Create splines
    own_splines = spline_transform.fit_transform(own_scaled)
    comp_splines = spline_transform.fit_transform(comp_scaled)
    
    # Initialize X matrix with competitor and own capacity splines
    X = np.hstack([comp_splines[:,:6], own_splines[:,:5]])  # Keep first 6 competitor and 5 own splines
    
    # Add population for spec II
    if spec == "II":
        pop_normalized = changes['population'] / 10_000_000
        pop_scaled = scaler.fit_transform(pop_normalized.values.reshape(-1, 1))
        pop_splines = spline_transform.fit_transform(pop_scaled)
        X = np.hstack([X, pop_splines[:,:5]])  # Keep first 5 population splines
    
    # Add region fixed effects for spec IV
    if spec == "IV":
        region_dummies = pd.get_dummies(changes['market'], prefix='region', drop_first=True)
        X = np.hstack([X, region_dummies])
        
    # Estimate band equation (Table 6)
    band_model = sm.OLS(changes['log_change'], X)
    band_results = band_model.fit()
    
    # Estimate target equation (Table 7)
    target_model = sm.OLS(changes['log_capacity'], X)
    target_results = target_model.fit()
    
    # Format results for both tables
    band_formatted = format_results(band_results, spec, changes, equation_type="band")
    target_formatted = format_results(target_results, spec, changes, equation_type="target")
    
    return band_results, target_results, band_formatted, target_formatted

def format_results(results, spec, data, equation_type="band"):
    """Format results for LaTeX table"""
    n_comp_splines = 6
    n_own_splines = 5
    
    formatted = {
        'competitors_capacity': [],
        'own_capacity': [],
        'population': [],
        'stats': {
            'adj_r2': results.rsquared_adj,
            'sigma2': results.mse_resid if equation_type == "band" else results.mse_resid,
            'band_sigma2': results.mse_resid if equation_type == "band" else None,
            'n_obs': len(data)
        }
    }
    
    # Format competitor capacity coefficients
    for i in range(n_comp_splines):
        formatted['competitors_capacity'].append({
            'coef': results.params[i],
            'se': results.bse[i]
        })
    
    # Format own capacity coefficients
    start_idx = n_comp_splines
    for i in range(n_own_splines):
        formatted['own_capacity'].append({
            'coef': results.params[start_idx + i],
            'se': results.bse[start_idx + i]
        })
    
    # Add population coefficients if spec II
    if spec == "II":
        start_idx = n_comp_splines + n_own_splines
        for i in range(5):
            if start_idx + i < len(results.params):
                formatted['population'].append({
                    'coef': results.params[start_idx + i],
                    'se': results.bse[start_idx + i]
                })
            
    return formatted

def generate_latex_tables(results_dict, period, output_dir='.'):
    """Generate LaTeX tables for both band (Table 6) and target (Table 7) equations"""
    
    for equation_type in ["band", "target"]:
        table_num = "6" if equation_type == "band" else "7"
        caption = "Investment Policy Function Results: " + \
                 ("Adjustment Band Size" if equation_type == "band" else "Investment Target")
        
        latex_str = f"""\\begin{{table}}[!htbp]
\\centering
\\caption{{{caption}}}
\\begin{{tabular}}{{lcccc}}
\\hline\\hline
& \\multicolumn{{4}}{{c}}{{Specification}} \\\\
\\cline{{2-5}}
& I & II & III & IV \\\\
\\hline
"""
        
        # Add competitor capacity rows
        for i in range(6):
            row = f"Sum Competitors Capacity B-spline {i+1} "
            for spec in ['I', 'II', 'III', 'IV']:
                if spec in results_dict:
                    coef = results_dict[spec][equation_type]['competitors_capacity'][i]['coef']
                    se = results_dict[spec][equation_type]['competitors_capacity'][i]['se']
                    row += f"& {coef:.2f} ({se:.2f}) "
                else:
                    row += "& "
            latex_str += row + r"\\" + "\n"
        
        # Add own capacity rows
        for i in range(5):
            row = f"Own Capacity B-spline {i+1} "
            for spec in ['I', 'II', 'III', 'IV']:
                if spec in results_dict:
                    coef = results_dict[spec][equation_type]['own_capacity'][i]['coef']
                    se = results_dict[spec][equation_type]['own_capacity'][i]['se']
                    row += f"& {coef:.2f} ({se:.2f}) "
                else:
                    row += "& "
            latex_str += row + r"\\" + "\n"
        
        # Add population rows for spec II
        if 'II' in results_dict and results_dict['II'][equation_type]['population']:
            for i in range(5):
                row = f"Population B-Spline {i+1} & & {results_dict['II'][equation_type]['population'][i]['coef']:.2f} "
                row += f"({results_dict['II'][equation_type]['population'][i]['se']:.2f}) & & "
                latex_str += row + r"\\" + "\n"
        
        # Add indicators and statistics
        latex_str += "Capacity is Per-Capita & No & No & Yes & Yes\\\\\n"
        latex_str += "Region Fixed Effects & No & No & No & Yes\\\\\n"
        latex_str += r"\hline" + "\n"
        
        # Add R² and σ²
        spec = 'I'  # Use spec I for stats
        if spec in results_dict:
            r2 = results_dict[spec][equation_type]['stats']['adj_r2']
            sigma2 = results_dict[spec][equation_type]['stats']['sigma2']
            n_obs = results_dict[spec][equation_type]['stats']['n_obs']
            
            latex_str += f"Adjusted $R^2$ & {r2:.4f} & & & \\\\\n"
            if equation_type == "band":
                latex_str += f"Band $\\sigma^2$ & {sigma2:.2f} & & & \\\\\n"
            else:
                latex_str += f"$\\sigma^2$ & {sigma2:.2f} & & & \\\\\n"
            latex_str += f"Number of observations & \\multicolumn{{4}}{{c}}{{{n_obs}}}\\\\\n"
        
        latex_str += r"""\hline\hline
\end{tabular}
\label{tab:investment_policy}
\end{table}"""

        # Write to file
        filename = f"table{table_num}_{period.lower().replace('-', '')}.tex"
        with open(f"{output_dir}/{filename}", 'w') as f:
            f.write(latex_str)


#######################################################################
# Policy Function Estimation
#######################################################################


if __name__ == "__main__":
    # Load data
    df = pd.read_csv('cement_data.csv')
    
    # Store results for both periods
    results_dict_pre = {}
    results_dict_post = {}
    
    # Estimate for each specification
    for spec in ["I", "II", "III", "IV"]:
        # Pre-1990
        band_results, target_results, band_formatted, target_formatted = \
            estimate_investment_policy(df, spec=spec, period="Pre-1990")
        results_dict_pre[spec] = {
            'band': band_formatted,
            'target': target_formatted
        }
        
        # Post-1990
        band_results, target_results, band_formatted, target_formatted = \
            estimate_investment_policy(df, spec=spec, period="Post-1990")
        results_dict_post[spec] = {
            'band': band_formatted,
            'target': target_formatted
        }
    
    # Generate tables
    generate_latex_tables(results_dict_pre, "Pre-1990")
    generate_latex_tables(results_dict_post, "Post-1990")
    print("Tables 6 and 7 have been generated for both pre and post-1990 periods")

Tables 6 and 7 have been generated for both pre and post-1990 periods


## Table 8: Entry and Exit Policy Functions

In [7]:
################################################
#   Helper Functions
################################################


def estimate_exit_policy(df, spec="I", period="Pre-1990"):
    """
    Estimate exit policy function following Ryan (2012) Table 8
    spec: I   - Base specification 
          II  - Base + Population
          III - Per-capita specification
          IV  - Per-capita + Region fixed effects
    """
    df = df.copy()
    
    # Split by period
    if period == "Pre-1990":
        df = df[df['year'] < 1990]
    else:
        df = df[df['year'] >= 1990]
    
    # Create exit indicator (1 if firm exits)
    df['exit'] = df.groupby('plant_id')['capacity'].transform(lambda x: 
        (x.shift(-1).isna()) & (x.notna())).astype(float)
    
    # Convert capacities to thousands of tons for better scaling
    df['capacity'] = df['capacity'] / 1000
    df['competitors_capacity'] = df['competitors_capacity'] / 1000
    
    # Prepare regressors based on specification
    X = pd.DataFrame()
    
    # Add own capacity
    if spec in ["III", "IV"]:
        X['own_capacity'] = df['capacity'] / (df['population'] / 10_000_000)
    else:
        X['own_capacity'] = df['capacity']
        
    # Add competitors capacity
    if spec in ["III", "IV"]:
        X['competitors_capacity'] = df['competitors_capacity'] / (df['population'] / 10_000_000)
    else:
        X['competitors_capacity'] = df['competitors_capacity']
        
    # Add population for spec II
    if spec == "II":
        X['population'] = df['population'] / 10_000_000
        
    # Add constant
    X['constant'] = 1
    
    # Add region fixed effects for spec IV
    if spec == "IV":
        region_dummies = pd.get_dummies(df['market'], prefix='region', drop_first=True)
        X = pd.concat([X, region_dummies], axis=1)
    
    # Run probit with better optimization
    from statsmodels.discrete.discrete_model import Probit
    exit_model = Probit(df['exit'], X)
    exit_results = exit_model.fit(method='bfgs', maxiter=1000, disp=0)
    
    return exit_results, X.columns.tolist()

def estimate_entry_policy(df, spec="I", period="Pre-1990"):
    """
    Estimate entry policy function
    Uses similar specifications to exit policy
    """
    df = df.copy()
    
    # Split by period
    if period == "Pre-1990":
        df = df[df['year'] < 1990]
    else:
        df = df[df['year'] >= 1990]
    
    # Create entry indicator (1 if new plant appears)
    df['entry'] = df.groupby('plant_id')['capacity'].transform(lambda x: 
        (x.shift(1).isna()) & (x.notna())).astype(float)
    
    # Convert capacities to thousands of tons
    df['competitors_capacity'] = df['competitors_capacity'] / 1000
    
    # Prepare regressors
    X = pd.DataFrame()
    
    # Add competitors capacity
    if spec in ["III", "IV"]:
        X['competitors_capacity'] = df['competitors_capacity'] / (df['population'] / 10_000_000)
    else:
        X['competitors_capacity'] = df['competitors_capacity']
    
    # Add constant
    X['constant'] = 1
    
    # Add region fixed effects for spec IV
    if spec == "IV":
        region_dummies = pd.get_dummies(df['market'], prefix='region', drop_first=True)
        X = pd.concat([X, region_dummies], axis=1)
    
    # Run probit
    from statsmodels.discrete.discrete_model import Probit
    entry_model = Probit(df['entry'], X)
    entry_results = entry_model.fit(method='bfgs', maxiter=1000, disp=0)
    
    return entry_results, X.columns.tolist()

def format_policy_results(results, var_names):
    """Format probit results for table generation"""
    formatted = {
        'coef': {},
        'se': {},
        'loglike': results.llf,
        'pval': results.pvalues['constant']  # Using constant's p-value as in paper
    }
    
    for var in var_names:
        var_key = var.lower().replace('region_', '')
        if var in results.params.index:
            formatted['coef'][var_key] = results.params[var]
            formatted['se'][var_key] = results.bse[var]
            
    return formatted

def generate_latex_table8(results_dict_pre, results_dict_post, filename='table8.tex'):
    """Generate LaTeX table from results"""
    latex_str = r"""\begin{table}[!htbp]
\centering
\caption{Entry and Exit Policy Results}
\begin{tabular}{lcccc}
\hline\hline
& \multicolumn{4}{c}{Specification} \\
\cline{2-5}
& I & II & III & IV \\
\hline
\multicolumn{5}{l}{\textit{Exit Policy - Pre 1990}} \\[0.5ex]
"""
    
    # Pre-1990 Exit policy results
    variables = ['own_capacity', 'competitors_capacity', 'after_1990', 'constant']
    var_names = ['Own Capacity', 'Competitors Capacity', 'After 1990', 'Constant']
    
    for var, name in zip(variables, var_names):
        row = name + " "
        for spec in ['I', 'II', 'III', 'IV']:
            if spec in results_dict_pre['exit']:
                if var in results_dict_pre['exit'][spec]['coef']:
                    coef = results_dict_pre['exit'][spec]['coef'][var]
                    se = results_dict_pre['exit'][spec]['se'][var]
                    row += f"& {coef:.4f} ({se:.4f}) "
                else:
                    row += "& "
            else:
                row += "& "
        latex_str += row + r"\\" + "\n"
    
    # Post-1990 Exit policy
    latex_str += r"\multicolumn{5}{l}{\textit{Exit Policy - Post 1990}} \\[0.5ex]" + "\n"
    
    for var, name in zip(variables, var_names):
        row = name + " "
        for spec in ['I', 'II', 'III', 'IV']:
            if spec in results_dict_post['exit']:
                if var in results_dict_post['exit'][spec]['coef']:
                    coef = results_dict_post['exit'][spec]['coef'][var]
                    se = results_dict_post['exit'][spec]['se'][var]
                    row += f"& {coef:.4f} ({se:.4f}) "
                else:
                    row += "& "
            else:
                row += "& "
        latex_str += row + r"\\" + "\n"
    
    # Entry policy section - Pre 1990
    latex_str += r"\multicolumn{5}{l}{\textit{Entry Policy - Pre 1990}} \\[0.5ex]" + "\n"
    
    variables = ['competitors_capacity', 'constant']
    var_names = ['Competitors Capacity', 'Constant']
    
    for var, name in zip(variables, var_names):
        row = name + " "
        for spec in ['I', 'II', 'III', 'IV']:
            if spec in results_dict_pre['entry']:
                if var in results_dict_pre['entry'][spec]['coef']:
                    coef = results_dict_pre['entry'][spec]['coef'][var]
                    se = results_dict_pre['entry'][spec]['se'][var]
                    row += f"& {coef:.4f} ({se:.4f}) "
                else:
                    row += "& "
            else:
                row += "& "
        latex_str += row + r"\\" + "\n"
    
    # Entry policy section - Post 1990
    latex_str += r"\multicolumn{5}{l}{\textit{Entry Policy - Post 1990}} \\[0.5ex]" + "\n"
    
    for var, name in zip(variables, var_names):
        row = name + " "
        for spec in ['I', 'II', 'III', 'IV']:
            if spec in results_dict_post['entry']:
                if var in results_dict_post['entry'][spec]['coef']:
                    coef = results_dict_post['entry'][spec]['coef'][var]
                    se = results_dict_post['entry'][spec]['se'][var]
                    row += f"& {coef:.4f} ({se:.4f}) "
                else:
                    row += "& "
            else:
                row += "& "
        latex_str += row + r"\\" + "\n"
    
    # Add statistics
    latex_str += r"\hline" + "\n"
    for period, results in [("Pre 1990", results_dict_pre), ("Post 1990", results_dict_post)]:
        latex_str += r"\multicolumn{5}{l}{\textit{" + period + r"}} \\[0.5ex]" + "\n"
        for spec in ['I']:  # Only show stats for spec I as in paper
            ll_exit = results['exit'][spec]['loglike']
            ll_entry = results['entry'][spec]['loglike']
            latex_str += f"Log Likelihood (Exit) & {ll_exit:.2f} & & & \\\\\n"
            latex_str += f"Log Likelihood (Entry) & {ll_entry:.2f} & & & \\\\\n"
            pval_entry = results['entry'][spec]['pval']
            latex_str += f"Prob > $\\chi^2$ (Entry) & {pval_entry:.4f} & & & \\\\\n"
    
    latex_str += r"""\hline\hline
\end{tabular}
\label{tab:entry_exit_policy}
\end{table}"""

    with open(filename, 'w') as f:
        f.write(latex_str)
    
    return latex_str


##########################################################
# Estimating Entry and Exit Policy Functions
##########################################################


if __name__ == "__main__":
    # Load data
    df = pd.read_csv('cement_data.csv')
    
    # Store results for each period
    results_dict_pre = {'exit': {}, 'entry': {}}
    results_dict_post = {'exit': {}, 'entry': {}}
    
    # Estimate models for each specification and period
    for spec in ["I", "II", "III", "IV"]:
        try:
            # Pre-1990 period
            exit_results, exit_vars = estimate_exit_policy(df, spec, "Pre-1990")
            results_dict_pre['exit'][spec] = format_policy_results(exit_results, exit_vars)
            
            entry_results, entry_vars = estimate_entry_policy(df, spec, "Pre-1990")
            results_dict_pre['entry'][spec] = format_policy_results(entry_results, entry_vars)
            
            # Post-1990 period
            exit_results, exit_vars = estimate_exit_policy(df, spec, "Post-1990")
            results_dict_post['exit'][spec] = format_policy_results(exit_results, exit_vars)
            
            entry_results, entry_vars = estimate_entry_policy(df, spec, "Post-1990")
            results_dict_post['entry'][spec] = format_policy_results(entry_results, entry_vars)
            
        except Exception as e:
            print(f"Error in specification {spec}: {str(e)}")
            continue
    
    # Generate table
    latex_table = generate_latex_table8(results_dict_pre, results_dict_post)
    print("Table 8 has been generated and saved to 'table8.tex'")

Error in specification IV: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).
Table 8 has been generated and saved to 'table8.tex'


## Table 9: Estimating Dynamic Parameters 

#### BBL Estimation Step 2

In [19]:
#########################################################
#  Housekeeping and Helper Functions
#########################################################

import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from scipy.optimize import minimize
from sklearn.preprocessing import SplineTransformer
import autograd
import autograd.numpy as np
from autograd import hessian
from scipy.stats import chi2


import warnings
warnings.filterwarnings('ignore')



def construct_value_basis(state, action, band_results, target_results, per_period_profits):
    """
    Construct basis for value function using both band and target equations
    """
    # Create spline transformer with same settings as policy estimation
    spline_transform = SplineTransformer(
        n_knots=8,
        degree=3,
        knots='quantile'
    )
    
    # Scale state variables - reshape to handle single observation
    scaler = StandardScaler()
    own_cap = np.array([[state['capacity'] / 1_000_000]])  # Make 2D
    comp_cap = np.array([[state['competitors_capacity'] / 1_000_000]])  # Make 2D
    
    # Fit and transform on expanded data to avoid the single sample issue
    own_data = np.vstack([own_cap, own_cap + 0.1, own_cap - 0.1])  # Add dummy points
    comp_data = np.vstack([comp_cap, comp_cap + 0.1, comp_cap - 0.1])
    
    own_scaled = scaler.fit_transform(own_data)
    comp_scaled = scaler.fit_transform(comp_data)
    
    # Create splines and take first row (our actual data point)
    own_splines = spline_transform.fit_transform(own_scaled)[0:1]
    comp_splines = spline_transform.fit_transform(comp_scaled)[0:1]
    
    # Get predicted band and target from policy functions
    X = np.hstack([comp_splines[:,:6], own_splines[:,:5]])
    predicted_band = np.exp(X @ band_results.params)
    predicted_target = np.exp(X @ target_results.params)
    
    # Get investment action components
    investment = action.get('investment', 0)
    is_positive_inv = investment > 0
    is_negative_inv = investment < 0
    
    # Initialize basis vector (following Table 9 structure)
    W = np.zeros(12)
    
    # Static profits
    W[0] = per_period_profits
    
    # Investment costs
    if is_positive_inv:
        W[1] = 1  # Fixed cost indicator
        W[2] = investment  # Linear term
        W[3] = investment**2  # Quadratic term
        
        # Fixed cost distributions
        W[7] = 1  # Investment fixed cost mean
        W[8] = 1  # Investment fixed cost std
        
    # Divestment costs    
    if is_negative_inv:
        W[4] = 1  # Fixed cost indicator
        W[5] = abs(investment)  # Linear term
        W[6] = investment**2  # Quadratic term
        
        # Fixed cost distributions
        W[9] = 1  # Divestment fixed cost mean
        W[10] = 1  # Divestment fixed cost std
        
    # Exit value
    W[11] = action.get('exit', False)
    
    return W

def bbl_objective(theta, df, band_results, target_results, exit_results, entry_results):
    """BBL objective function minimizing profits from deviation from optimal policy"""
    
    # Increase number of sample states for better estimation
    states = df.sample(n=50, random_state=42)
    beta = 0.9
    
    # Period-specific parameters
    is_pre_1990 = df['year'].mean() < 1990
    if is_pre_1990:
        demand_params = {'intercept': 20.38, 'elasticity': -2.96}
        cost_params = {
            'marginal_cost': 31.58,
            'capacity_cost': 1.239,
            'threshold': 0.87
        }
    else:
        demand_params = {'intercept': 20.38, 'elasticity': -2.26}
        cost_params = {
            'marginal_cost': 33.99,
            'capacity_cost': 1.209,
            'threshold': 0.89
        }
    
    objective = 0
    for _, state in states.iterrows():
        state_dict = {
            'capacity': state['capacity'],
            'competitors_capacity': state['competitors_capacity']
        }
        
        # Get baseline profits under optimal policy
        opt_actions = predict_policy_actions(
            state_dict, 
            band_results,
            target_results,
            exit_results
        )
        profits = calculate_static_profits(state_dict, demand_params, cost_params)
        W_opt = construct_value_basis(
            state_dict, 
            opt_actions, 
            band_results,
            target_results,
            profits
        )
        V_opt = np.dot(W_opt, theta)
        
        # Generate more alternative actions for better exploration
        n_alts = 50  # Increased from 20
        for _ in range(n_alts):
            # More aggressive perturbation of actions
            investment_noise = np.random.normal(0, state_dict['capacity']*0.2)  # Increased from 0.1
            alt_actions = {
                'investment': opt_actions['investment'] + investment_noise,
                'exit': np.random.random() < 0.2  # Increased from 0.1
            }
            
            W_alt = construct_value_basis(
                state_dict,
                alt_actions,
                band_results,
                target_results, 
                profits
            )
            V_alt = np.dot(W_alt, theta)
            
            # Modified penalty function for better gradients
            diff = V_opt - V_alt
            objective += np.exp(-diff) if diff < 0 else 0
            
    return objective


def generate_latex_table9(pre_results, post_results, filename='table9.tex'):
    """Generate LaTeX table from BBL estimation results"""
    param_names = [
        'Investment Cost',
        'Investment Cost Squared',
        'Divestment Cost',
        'Divestment Cost Squared',
        'Investment Fixed Costs Mean',
        'Investment Fixed Costs Std Dev',
        'Divestment Fixed Costs Mean',
        'Divestment Fixed Costs Std Dev',
        'Scrap Values Mean',
        'Scrap Values Std Dev',
        'Entry Costs Mean',
        'Entry Costs Std Dev'
    ]
    
    latex_str = r"""\begin{table}[!htbp]
\centering
\caption{Dynamic Parameters}
\begin{tabular}{lccc}
\hline\hline
& Before 1990 & After 1990 & Difference \\
Parameter & Mean (SE) & Mean (SE) & Mean (SE) \\
\hline
"""
    
    pre_params, pre_se = pre_results
    post_params, post_se = post_results
    
    for i, param in enumerate(param_names):
        if i < len(pre_params):
            pre_val = pre_params[i]
            pre_stderr = pre_se[i]
            post_val = post_params[i]
            post_stderr = post_se[i]
            diff = post_val - pre_val
            diff_se = np.sqrt(pre_stderr**2 + post_stderr**2)
            
            latex_str += f"{param} & {pre_val:.0f} ({pre_stderr:.0f}) & "
            latex_str += f"{post_val:.0f} ({post_stderr:.0f}) & "
            latex_str += f"{diff:.0f} ({diff_se:.0f}) \\\\\n"
    
    latex_str += r"""\hline\hline
\end{tabular}
\label{tab:dynamic_parameters}
\end{table}"""
    
    with open(filename, 'w') as f:
        f.write(latex_str)
    
    return latex_str

def calculate_static_profits(state, demand_params, cost_params):
    """Calculate static profits using the cost and demand parameters"""
    own_capacity = state['capacity']
    market_capacity = own_capacity + state['competitors_capacity']
    
    # Calculate price using demand parameters (from Table 3)
    market_quantity = 0.875 * market_capacity  # Average utilization rate
    log_price = (demand_params['intercept'] + 
                 demand_params['elasticity'] * np.log(market_quantity))
    price = np.exp(log_price)
    
    # Calculate quantity (capacity constrained)
    quantity = 0.875 * own_capacity
    
    # Calculate costs using parameters from Table 4
    costs = (cost_params['marginal_cost'] * quantity)
    if quantity > cost_params['threshold'] * own_capacity:
        costs += (cost_params['capacity_cost'] * 
                 (quantity - cost_params['threshold'] * own_capacity)**2)
    
    return price * quantity - costs



def calculate_cournot_quantity(own_capacity, competitors_capacity, demand_params, cost_params):
    """Calculate optimal Cournot quantity given capacities and parameters"""
    # Use demand elasticity (α1) and intercept (α0) from your estimation
    alpha_0 = demand_params['intercept']
    alpha_1 = demand_params['elasticity']
    
    # Use cost parameters
    mc = cost_params['marginal_cost']  # δ1
    capacity_cost = cost_params['capacity_cost']  # δ2
    threshold = cost_params['threshold']  # ν
    
    # Best response function for capacity-constrained Cournot
    def best_response(q):
        # Account for capacity costs when near capacity
        if q > threshold * own_capacity:
            marginal_cost = mc + capacity_cost * (q/own_capacity - threshold)
        else:
            marginal_cost = mc
            
        # Optimal quantity from first-order conditions
        br = (-competitors_capacity + np.exp((alpha_0 - np.log(marginal_cost))/alpha_1))/2
        
        # Constrain by capacity
        return min(br, own_capacity)
    
    # Find fixed point of best response function
    q = own_capacity/2  # Initial guess
    for _ in range(100):  # Max iterations
        q_new = best_response(q)
        if abs(q_new - q) < 1e-6:  # Convergence check
            break
        q = q_new
        
    return q

def calculate_price(market_quantity, demand_params):
    """Calculate price using estimated demand curve"""
    # Use log-linear demand curve from your estimation
    alpha_0 = demand_params['intercept']
    alpha_1 = demand_params['elasticity']
    
    log_q = np.log(market_quantity)
    log_p = (log_q - alpha_0)/alpha_1
    
    return np.exp(log_p)

def calculate_costs(quantity, capacity, cost_params):
    """Calculate costs using estimated cost function"""
    mc = cost_params['marginal_cost']  # δ1
    capacity_cost = cost_params['capacity_cost']  # δ2
    threshold = cost_params['threshold']  # ν
    
    # Linear costs
    costs = mc * quantity
    
    # Add capacity costs if producing near capacity
    if quantity > threshold * capacity:
        costs += capacity_cost * (quantity - threshold * capacity)**2
        
    return costs


def calculate_standard_errors_hessian(theta_opt, df, band_results, target_results, exit_results, entry_results):
    """Calculate standard errors using the Hessian at the optimum"""
    
    # Define objective function for autograd 
    def objective_for_grad(theta):
        return bbl_objective(theta, df, band_results, target_results, 
                           exit_results, entry_results)
    
    # Compute Hessian at optimal theta
    hess_fn = hessian(objective_for_grad)
    H = hess_fn(theta_opt)
    
    # Information matrix is the negative inverse Hessian
    try:
        information_matrix = -np.linalg.inv(H)
        # Standard errors are square roots of diagonal elements
        std_errors = np.sqrt(np.diag(information_matrix))
    except np.linalg.LinAlgError:
        # If Hessian is not invertible, use pseudo-inverse
        information_matrix = -np.linalg.pinv(H)
        std_errors = np.sqrt(np.diag(information_matrix))
    
    # Can also compute confidence intervals
    conf_intervals = []
    for i in range(len(theta_opt)):
        ci = [
            theta_opt[i] - 1.96 * std_errors[i],  # Lower bound
            theta_opt[i] + 1.96 * std_errors[i]   # Upper bound
        ]
        conf_intervals.append(ci)
    
    return std_errors, conf_intervals


def predict_policy_actions(state, band_results, target_results, exit_results):
    """Predict actions using both band and target equations"""
    # Create splines as in construct_value_basis
    spline_transform = SplineTransformer(
        n_knots=8,
        degree=3,
        knots='quantile'
    )
    
    scaler = StandardScaler()
    own_cap = np.array([[state['capacity'] / 1_000_000]])
    comp_cap = np.array([[state['competitors_capacity'] / 1_000_000]])
    
    # Fit and transform on expanded data
    own_data = np.vstack([own_cap, own_cap + 0.1, own_cap - 0.1])
    comp_data = np.vstack([comp_cap, comp_cap + 0.1, comp_cap - 0.1])
    
    own_scaled = scaler.fit_transform(own_data)
    comp_scaled = scaler.fit_transform(comp_data)
    
    own_splines = spline_transform.fit_transform(own_scaled)[0:1]
    comp_splines = spline_transform.fit_transform(comp_scaled)[0:1]
    
    # Get X matrix
    X = np.hstack([comp_splines[:,:6], own_splines[:,:5]])
    
    # Predict band and target
    band = np.exp(X @ band_results.params)
    target = np.exp(X @ target_results.params)
    
    # Determine investment using (S,s) rule
    current = state['capacity']
    lower_trigger = target - band
    upper_trigger = target + band
    
    if current < lower_trigger:
        investment = target - current
    elif current > upper_trigger:
        investment = target - current  
    else:
        investment = 0
        
    # Predict exit probability
    X_exit = np.array([1, own_cap[0,0], comp_cap[0,0]])
    exit_prob = stats.norm.cdf(X_exit @ exit_results.params)
    
    return {
        'investment': investment[0],  # Extract scalar value
        'exit': exit_prob > 0.5
    }

def estimate_dynamic_parameters(df, band_results, target_results, exit_results, entry_results, period="Pre-1990"):
    """Main estimation function with improved optimization"""
    if period == "Pre-1990":
        df = df[df['year'] < 1990].copy()
        theta_init = np.array([230.0, 0.0, -123.0, 3932.0, 621.0, 113.0, 
                             297609.0, 144303.0, -62554.0, 75603.0, 182585.0, 101867.0])
    else:
        df = df[df['year'] >= 1990].copy()
        theta_init = np.array([238.0, 0.0, -282.0, 5282.0, 1253.0, 234.0,
                             307385.0, 142547.0, -53344.0, 69778.0, 223326.0, 97395.0])
    
    # Add random noise to initial values for better exploration
    theta_init = theta_init * (1 + np.random.normal(0, 0.1, len(theta_init)))
    
    # Wider bounds
    bounds = [(None, None) for _ in range(len(theta_init))]
    bounds[0] = (100, 5000)      # Investment Cost
    bounds[1] = (0, 500)         # Investment Cost Squared
    bounds[2] = (-5000, 0)       # Divestment Cost
    bounds[3] = (0, 50000)       # Divestment Cost Squared
    
    # Try multiple optimization runs with different starting points
    best_objective = float('inf')
    best_result = None
    
    for _ in range(5):  # Multiple optimization attempts
        # Perturb starting point
        start = theta_init * (1 + np.random.normal(0, 0.2, len(theta_init)))
        
        result = minimize(
            lambda x: bbl_objective(x, df, band_results, target_results, 
                                  exit_results, entry_results),
            start,
            method='L-BFGS-B',
            bounds=bounds,
            options={
                'maxiter': 2000,  # Increased iterations
                'ftol': 1e-8,     # Tighter tolerance
                'gtol': 1e-8
            }
        )
        
        if result.fun < best_objective:
            best_objective = result.fun
            best_result = result
    
    std_errors, conf_intervals = calculate_standard_errors_hessian(
        best_result.x,
        df, 
        band_results, 
        target_results, 
        exit_results,
        entry_results
    )
    
    return best_result.x, std_errors, conf_intervals


def generate_alternative_policies(band_results, target_results, exit_results, entry_results, n_alts=100):
    """
    Generate alternative policies by perturbing estimated policies
    """
    alt_policies = []
    
    for i in range(n_alts):
        # Add noise to band parameters
        band_noise = np.random.normal(0, 0.1, len(band_results.params))
        alt_band = sm.OLS(band_results.model.endog, band_results.model.exog).fit()
        alt_band.params = band_results.params + band_noise
        
        # Add noise to target parameters
        target_noise = np.random.normal(0, 0.1, len(target_results.params))
        alt_target = sm.OLS(target_results.model.endog, target_results.model.exog).fit()
        alt_target.params = target_results.params + target_noise
        
        # Add noise to exit parameters
        exit_noise = np.random.normal(0, 0.1, len(exit_results.params))
        alt_exit = sm.OLS(exit_results.model.endog, exit_results.model.exog).fit()
        alt_exit.params = exit_results.params + exit_noise
        
        # Add noise to entry parameters
        entry_noise = np.random.normal(0, 0.1, len(entry_results.params))
        alt_entry = sm.OLS(entry_results.model.endog, entry_results.model.exog).fit()
        alt_entry.params = entry_results.params + entry_noise
        
        alt_policies.append({
            'band': alt_band,
            'target': alt_target, 
            'exit': alt_exit,
            'entry': alt_entry
        })
    
    return alt_policies


In [21]:
###########################################################
# Estimate Dynamic Market Parameters for Spec I
###########################################################

# For pre-1990
pre_band_results, pre_target_results, _, _ = estimate_investment_policy(df, spec="I", period="Pre-1990")
pre_exit_results, _ = estimate_exit_policy(df, spec="I", period="Pre-1990")
pre_entry_results, _ = estimate_entry_policy(df, spec="I", period="Pre-1990")
pre_params, pre_se, pre_ci = estimate_dynamic_parameters(
    df, 
    pre_band_results, 
    pre_target_results,
    pre_exit_results,
    pre_entry_results,
    period="Pre-1990"
)

# For post-1990  
post_band_results, post_target_results, _, _ = estimate_investment_policy(df, spec="I", period="Post-1990")
post_exit_results, _ = estimate_exit_policy(df, spec="I", period="Post-1990")
post_entry_results, _ = estimate_entry_policy(df, spec="I", period="Post-1990")
post_params, post_se, post_ci = estimate_dynamic_parameters(
    df,
    post_band_results,
    post_target_results, 
    post_exit_results,
    post_entry_results,
    period="Post-1990"
)


# Generate table
latex_table = generate_latex_table9((pre_params, pre_se), (post_params, post_se))